<a href="https://colab.research.google.com/github/akhargha/ShelfAware/blob/main/ML/fine_tuning_tutorial_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# 1. Data Preparation
data_dir = '/content/data/few_shot/'

# Configure data augmentation for training
datagen = ImageDataGenerator(
    rotation_range=20,        # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,    # Randomly shift images horizontally
    height_shift_range=0.2,   # Randomly shift images vertically
    shear_range=0.15,         # Randomly apply shear transformations
    zoom_range=0.2,           # Randomly zoom in on images
    horizontal_flip=True,     # Randomly flip images horizontally
    rescale=1.0/255           # Normalize pixel values to the range [0, 1]
)

# Load training data
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),   # Resize images to 224x224
    batch_size=16,            # Use a batch size of 16
    class_mode='categorical'  # Use categorical labels
)

# Check class labels
class_labels = list(train_data.class_indices.keys())
print(f"Class Labels: {class_labels}")

# 2. Model Construction
# Load the base MobileNetV2 model with ImageNet weights, excluding the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False)
base_model.trainable = False  # Freeze the base model layers

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add a global average pooling layer
x = Dropout(0.2)(x)              # Add dropout for regularization
x = Dense(128, activation='relu')(x)  # Add a fully connected layer
output = Dense(train_data.num_classes, activation='softmax')(x)  # Output layer for classification

# Define the final model
model = Model(inputs=base_model.input, outputs=output)

# 3. Model Compilation and Training
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Use Adam optimizer with a low learning rate
    loss='categorical_crossentropy',      # Use categorical crossentropy as the loss function
    metrics=['accuracy']                  # Track accuracy during training
)

# Configure early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(
    train_data,
    epochs=50,  # Train for a maximum of 50 epochs
    steps_per_epoch=train_data.samples // train_data.batch_size,  # Number of steps per epoch
    validation_data=train_data,  # Use the training data for validation
    validation_steps=train_data.samples // train_data.batch_size,  # Number of validation steps
    callbacks=[early_stopping]  # Use early stopping during training
)

# 4. Save the Model
model.save('/content/fine_tuned_model.h5')  # Save the fine-tuned model

# 5. Test Prediction
# Load the saved fine-tuned model
fine_tuned_model = tf.keras.models.load_model('/content/fine_tuned_model.h5')

# Load a test image and preprocess it
test_image_path = '/content/data/test_image.jpg'
img = image.load_img(test_image_path, target_size=(224, 224))  # Resize the image to 224x224
img_array = np.expand_dims(image.img_to_array(img), axis=0) / 255.0  # Normalize pixel values and expand dimensions

# Make a prediction using the fine-tuned model
predictions = fine_tuned_model.predict(img_array)
predicted_class_index = np.argmax(predictions)  # Get the index of the class with the highest probability
predicted_class = class_labels[predicted_class_index]  # Map the index to the corresponding class label

# Output the predicted class
print(f"Predicted Class: {predicted_class}")

Found 5 images belonging to 3 classes.


<ipython-input-13-917c7d72fd92>:33: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.0000e+00 - loss: 2.0470 - val_accuracy: 0.4000 - val_loss: 0.9447
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0000e+00 - loss: 1.4070 - val_accuracy: 0.2000 - val_loss: 0.9391
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.4000 - loss: 1.0678 - val_accuracy: 0.4000 - val_loss: 1.0800
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4000 - loss: 0.9204 - val_accuracy: 0.4000 - val_loss: 1.0737
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4000 - loss: 0.7750 - val_accuracy: 0.8000 - val_loss: 0.6334
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.5795 - val_accuracy: 1.0000 - val_loss: 0.6160
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6000 - loss: 0.7183 - val_accuracy: 0.8000 - val_loss: 0.7052
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.4052 - val_accuracy: 1.0000 - val_loss: 0.4419
Epoch 9/

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: juice


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.applications import imagenet_utils
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 1. Load Fine-Tuned Model
fine_tuned_model_path = '/Users/lucashyun/Trinity/weshack/model/fine_tuned_model.h5'  # Path to the Fine-Tuned model
fine_tuned_model = tf.keras.models.load_model(fine_tuned_model_path)

# Labels for the Fine-Tuned model
fine_tuned_labels = ['beverage', 'juice']  # Class names used during Fine-Tuning

# 2. Initialize MobileNetV2 (ImageNet Pretrained Model)
imagenet_model = tf.keras.applications.MobileNetV2(weights='imagenet')

# 3. Load and Display Image
filename = '/Users/lucashyun/Trinity/weshack/detected_objects/cluster_0.png'  # Path to the test image
img = image.load_img(filename, target_size=(224, 224))  # Resize the image to 224x224
plt.imshow(img)
plt.axis('off')
plt.show()

# 4. Preprocess the Image
resizedimg = image.img_to_array(img)  # Convert the image to a NumPy array
finalimg = np.expand_dims(resizedimg, axis=0)  # Expand dimensions to match input shape
finalimg = tf.keras.applications.mobilenet_v2.preprocess_input(finalimg)  # Preprocess for ImageNet model

# 5. Make Predictions with MobileNetV2 (ImageNet)
imagenet_predictions = imagenet_model.predict(finalimg)
imagenet_results = imagenet_utils.decode_predictions(imagenet_predictions, top=3)

# 6. Make Predictions with Fine-Tuned Model
fine_tuned_predictions = fine_tuned_model.predict(finalimg / 255.0)  # Fine-Tuned model expects [0, 1] normalized data
fine_tuned_index = np.argmax(fine_tuned_predictions)
fine_tuned_label = fine_tuned_labels[fine_tuned_index]
fine_tuned_confidence = fine_tuned_predictions[0][fine_tuned_index]

# 7. Output Results
print(f"Image File: {filename}")

# Print predictions from ImageNet model
print("\n[ImageNet (Pretrained MobileNetV2) Predictions]")
for i, (imagenet_id, label, confidence) in enumerate(imagenet_results[0]):
    print(f"{i + 1}. {label} (Confidence: {confidence:.2f})")

# Print predictions from Fine-Tuned model
print("\n[Fine-Tuned Model Prediction]")
print(f"Predicted Label: {fine_tuned_label} (Confidence: {fine_tuned_confidence:.2f})")

# Final decision: Compare Fine-Tuned model and ImageNet model predictions
print("\n[Final Decision]")
if fine_tuned_confidence > 0.5:  # Prioritize Fine-Tuned model if confidence is above 0.5
    print(f"Result: {fine_tuned_label} (from Fine-Tuned Model)")
else:
    print(f"Result: {imagenet_results[0][0][1]} (from ImageNet Model)")
